In [ ]:
!pip install pylatexenc
!pip install qiskit
!pip install qiskit_aer
!pip install qiskit_ibm_runtime

In [1]:
import sys

from math import gcd

from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime import SamplerV2 as Sampler

In [2]:
!git clone https://github.com/jeanfredericlaprade/shor_qiskit.git

sys.path.insert(0,'/content/shor_qiskit')

from Qshor import Shor

Here we factorise the number $15$ using the Aer quantum computer simulator.

$15$ is the smallest "interesting" number that Shor's algorithm can factor.

In [ ]:
N = 15 
backend = AerSimulator()

shor = Shor(backend=backend)
result = shor.factor(N)

print(f"The list of factors of {N} as computed by the Shor's algorithm is {result.factors[0]}.")

The next cell prepares the quantum circuit and shows its principal components at a high level of abstraction.

There are three logically distinct quantum registers:
- the "up" register, which will be measured to obtain a quotient of the frequency we're looking for
- the "down" register, in which $a^x\,\text{mod}\,N$ is computed 
- the auxillary register, which serves as a temporary storage space for the calculation in the down register.

This circuit is an optimised implementation of the phase estimation algorithm applied to the operator that compute $a^x\,\text{mod}\,N$.

Two advanced subjects (for another time)!


In [ ]:
a = 13
circuit = shor.construct_circuit(N, a, measurement=True)
print("The circuit has a depth ", circuit.decompose(reps=4).depth(), " base operators")
circuit.draw('mpl')

In [6]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
sampler = Sampler(backend)
isa_circuit = pm.run(circuit)

result = sampler.run([isa_circuit]).result()

In [ ]:
counts = result[0].data.m.get_counts()
plot_histogram(counts)

The observed difference in likelyhood are caused by the small number of sample we took. The $4$ measured state are in fact equally probable.
The value we need to know to proceed is $r$ in 

$\Large \frac{m}{2^n} = \frac{s}{r}$

where $m$ is the measurement result, $s$ is a random number and $n$ is the number of qubit used in the "up" register, $8$ in our case.


To convert a binary representation to a decimal representation with little-endian convention:

> for a binary representation $m = b_{n-1} b_{n-2} \ldots b_1 b_0$, then
$m = \sum_{i=0}^{n-1} b_i*2^i$.

For example:

> $10010 = 2^1 + 2^4 = 18$

and for the basis state $01000000$:

> $\frac{m}{2^n} = \frac{2^6}{2^8} = \frac{1}{4}$

We therefore have found that $\tfrac{s}{r}$ can take four values: $0$, $0.25$, $0.5$ et $0.75$.

$r$ is the smallest common denominator of those numbers.

Find the value for $r$, and from it determine the factors of $15$.

In [ ]:
#What is the right value?
r = "<your answer>"

# a^(r/2) mod N
y = pow(a, r//2, N)

k = gcd(y - 1, N)

print(k)

Exercise: 
  Determine the number of qubits needed by this implementation of Shor's algorithm in relation to the size of the bit representation of $N$.
  
  Proceed by printing the circuit for a few different values of $N$.


In [ ]:
N = 35  # Modify this value, for example with N = 21 or N = 35.
nbit = N.bit_length()
print(f'Number of bits needed to encode {N}: {nbit}')

circuit = shor.construct_circuit(N, 2)
print(f"Number of qubits in Shor's circuit: {circuit.num_qubits}")

circuit.draw('mpl', scale=0.7)